In [1]:
import json
import re
from os.path import join
import os

import pandas as pd
import numpy as np
from tqdm import tqdm
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import optuna
from sklearn.metrics import mean_absolute_error
import shap
from sklearn.metrics import log_loss
from sklearn.metrics import precision_recall_curve, auc
import cfbd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

with open('../config.json') as f:
    configuration = cfbd.Configuration(
        access_token = json.load(f)['CFBD_API_KEY']
    )

In [2]:
YEARS = range(2013, 2025)
DATA_DIR = '../data'

In [3]:
def convert_to_snake_case(cols):
    cols_new = []
    for c in cols:
        s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', c)
        cols_new.append(re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower())
    return cols_new

# NOTE: use the advanced rushing stats

In [4]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.StatsApi(api_client)
    data = api_instance.get_advanced_game_stats(year=2013, week=1)
tmp = pd.DataFrame([val.to_dict() for val in data])
tmp.columns = convert_to_snake_case(tmp.columns)

import json; print(json.dumps(tmp.offense.values[0], indent=2))

{
  "passingPlays": {
    "explosiveness": 1.3828717946760372,
    "successRate": 0.37037037037037035,
    "totalPPA": 4.025595832919332,
    "ppa": 0.14909614195997523
  },
  "rushingPlays": {
    "explosiveness": 0.5885187332090218,
    "successRate": 0.38235294117647056,
    "totalPPA": -1.9341798213541586,
    "ppa": -0.056887641804534074
  },
  "passingDowns": {
    "explosiveness": 2.5592718260550753,
    "successRate": 0.21052631578947367,
    "ppa": 0.337070738054116
  },
  "standardDowns": {
    "explosiveness": 0.5917039039082818,
    "successRate": 0.4523809523809524,
    "ppa": -0.10268876217769124
  },
  "openFieldYardsTotal": 50,
  "openFieldYards": 1.4705882352941175,
  "secondLevelYardsTotal": 25,
  "secondLevelYards": 0.7352941176470589,
  "lineYardsTotal": 87,
  "lineYards": 2.5647058823529414,
  "stuffRate": 0.2647058823529412,
  "powerSuccess": 0.4,
  "explosiveness": 0.9338896294990284,
  "successRate": 0.3770491803278688,
  "totalPPA": 2.091416011565173,
  "ppa": 